In [121]:
import pandas as pd
import numpy as np
import pandas as pd
import sklearn.metrics.pairwise as pw 
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer 
from scipy import linalg, dot 
import csv

In [12]:
filename = 'E:/desk/abhudabi/sample.txt'
raw_data = open(filename, 'rb')
reader = csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_NONE)
x = list(reader)

print x


[['user1', 'bread', 'butter', 'jam'], ['user2', 'bread', 'butter', 'jam']]


In [8]:
usermap={}
userid=0
itemmap={}
itemid=0

In [14]:
for row in x:
    for i in range(len(row)):
        if i==0:
            if not usermap.has_key(row[i]):
                usermap[row[i]]=userid
                userid+=1
        else:
            if not itemmap.has_key(row[i]):
                itemmap[row[i]]=itemid
                itemid+=1
            
        


{'bread': 0, 'butter': 1, 'jam': 2}

In [46]:
#map users and items with increasing orders
    
itemdict={1:"television", 2:"player",3:"mobile",4:"pen"}    

In [4]:
#construct user item matrix

userItem=np.array([[1,1,1,0],[1,1,0,0],[1,1,1,0],[1,0,1,1]])

In [5]:
#find k nearest neighbour users
knn_model=NearestNeighbors(n_neighbors=2)
knn_model.fit(userItem)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=2, p=2, radius=1.0)

In [12]:
top_k_distances,top_k_users = knn_model.kneighbors(userItem, return_distance=True) 

In [14]:
#added 1 to avoid undefined error
top_k_distances=top_k_distances+1

#inversed the matrix to give more weight for more similar users
top_k_distances=1/top_k_distances
print top_k_distances
print top_k_users

[[ 1.          1.        ]
 [ 1.          2.        ]
 [ 1.          1.        ]
 [ 1.          2.41421356]]
[[0 2]
 [1 0]
 [0 2]
 [3 2]]


In [31]:
user_pred_k=np.zeros(userItem.shape)
for i in range(userItem.shape[0]): 
    user_pred_k[i,:] =   top_k_distances[i].dot(userItem[top_k_users[i]])

In [32]:
for i in range(user_pred_k.shape[0]):
    user_pred_k[i]=user_pred_k[i].argsort()[::-1]
    
    

[[ 2.          2.          2.          0.        ]
 [ 1.5         1.5         0.5         0.        ]
 [ 2.          2.          2.          0.        ]
 [ 1.41421356  0.41421356  1.41421356  1.        ]]
[2 1 0 3]
[1 0 2 3]
[[1 3 0 2]
 [3 0 1 2]
 [3 2 0 1]
 [3 0 1 2]]
[[ 2.  1.  0.  3.]
 [ 1.  0.  2.  3.]
 [ 2.  1.  0.  3.]
 [ 2.  0.  3.  1.]]


In [38]:
#Predict for user
user=1


In [48]:
getRecommendation(user_pred_k,user)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


['player', 'mobile']

In [47]:
def getRecommendation(user_pred_k,user):
    predictions=[]
    for itemnum in user_pred_k[user]:
        if userItem[user,itemnum]==0:
            predictions.append(itemdict[itemnum])
            
    return predictions         
        
    

In [24]:
#loading ratings data
mv_rating=pd.read_csv("E:/desk/Irecommend/data/ml-100k/u.data",sep="\t",header=None)
columns=["UserId","MovieId","Rating","TimeStamp"]
mv_rating.columns=columns

In [46]:
mv_rating=mv_rating.drop(["TimeStamp"],axis=1)
mv_rating

,UserId,MovieId,Rating
1,186,302,3
2,22,377,1


In [41]:
##loading movie details

mov_details=pd.read_csv("E:/desk/Irecommend/data/ml-100k/u.item",sep="|",header=None)
columns=["MovieId","MovieTitle","ReleaseDate","IMDbURL","Unknown","Action","Adventure","Animation","Children","Comedy","Crime","Documentary","Drama","Fantasy","FilmNoir","Horror","Musical","Mystery","Romance","SciFi","Thriller","War","Western"]
mov_details.columns=columns

In [42]:
mv_details=mov_details.drop(['MovieTitle',"ReleaseDate","IMDbURL"], axis=1)



In [48]:

rating=pd.merge(mv_rating,mv_details,how='left', on='MovieId')
rating.columns

Index([u'UserId', u'MovieId', u'Rating', u'Unknown', u'Action', u'Adventure',
       u'Animation', u'Children', u'Comedy', u'Crime', u'Documentary',
       u'Drama', u'Fantasy', u'FilmNoir', u'Horror', u'Musical', u'Mystery',
       u'Romance', u'SciFi', u'Thriller', u'War', u'Western'],
      dtype='object')

In [54]:
#loading web data
raw_data=pd.read_csv("E:/desk/Irecommend/data/websites/anonymous-msweb.test",header=None,skiprows=7)


In [55]:
user_activity = raw_data.loc[raw_data[0] != "A"] 

In [58]:
user_activity = user_activity.loc[:, :1] 
user_activity.columns = ['category','value'] 

In [61]:
print len(user_activity[user_activity['category'] =="V"].value.unique()) 
print len(user_activity[user_activity['category'] =="C"].value.unique()) 

236
5000


In [69]:
lastindex = user_activity.index[len(user_activity)-1]
lastindex

20484

In [79]:
for index,row in user_activity.iterrows(): 
    if(index <= lastindex ): 
        if(user_activity.loc[index,'category'] == "C"): 
            tmp = 0            
            userid = user_activity.loc[index,'value'] 
            user_activity.loc[index,'userid'] = userid 
            user_activity.loc[index,'webid'] = userid 
            tmp = userid 
            nextrow = True             
        elif(user_activity.loc[index,'category'] != "C" and nextrow == True): 
                webid = user_activity.loc[index,'value'] 
                user_activity.loc[index,'webid'] = webid 
                user_activity.loc[index,'userid'] = tmp 
                if(index != lastindex and user_activity.loc[index+1,'category'] == "C"): 
                    nextrow = False 
                    caseid = 0 

In [83]:
user_activity = user_activity[user_activity['category'] == "V" ] 
user_activity = user_activity[['userid','webid']] 


In [86]:
user_activity_sort = user_activity.sort('webid', ascending=True) 
user_activity_sort.head()

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,userid,webid
8022,11900.0,1000.0
18070,14410.0,1000.0
11340,12724.0,1000.0
13101,13148.0,1000.0
15461,13738.0,1000.0


In [87]:
sLength = len(user_activity_sort['webid'])

In [90]:
user_activity_sort['rating']=pd.Series(np.ones(sLength),index=user_activity.index)

In [91]:
ratmat = user_activity_sort.pivot(index='userid', columns='webid', values='rating').fillna(0) 

ratmat.head()

webid,1000.0,1001.0,1002.0,1003.0,1004.0,1005.0,1006.0,1007.0,1008.0,1009.0,...,1252.0,1253.0,1256.0,1257.0,1258.0,1261.0,1266.0,1278.0,1295.0,1297.0
userid,,,,,,,,,,,,,,,,,,,,,
10001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10002.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10003.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10004.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10005.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
print type(ratmat)
ratmat = ratmat.to_dense().as_matrix() 
print type(ratmat)

<class 'pandas.core.frame.DataFrame'>
<type 'numpy.ndarray'>


In [95]:
ratmat[1:5]


SyntaxError: invalid syntax (<ipython-input-95-30d02f01c018>, line 2)

In [103]:
df=pd.DataFrame()
df["k"]=[1,2,3]
print df
print df.to_dense().as_matrix()
print df.as_matrix()

   k
0  1
1  2
2  3
[[1]
 [2]
 [3]]
[[1]
 [2]
 [3]]


In [105]:
#item profile generation

items = raw_data.loc[raw_data[0] == "A"] 
items.columns = ['record','webid','vote','desc','url'] 

In [107]:
items = items[['webid','desc']] 
items.head()

,webid,desc
0,1277,NetShow for PowerPoint
1,1253,MS Word Development
2,1109,TechNet (World Wide Web Edition)
3,1038,SiteBuilder Network Membership
4,1205,Hardware Supprt


In [111]:
items2=items[items["webid"].isin(user_activity['webid'].tolist())]

In [112]:
items_sort = items2.sort('webid', ascending=True)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [114]:
v = TfidfVectorizer(stop_words ="english",max_features = 100,ngram_range= (0,3),sublinear_tf =True) 

x = v.fit_transform(items_sort['desc']) 


In [119]:

itemprof =x.todense()

In [122]:
#user profile creation

userprof = dot(ratmat,itemprof)/linalg.norm(ratmat)/linalg.norm(itemprof) 

In [129]:
userprof[1:3]

matrix([[ 0.00089668,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.00048815,  0.        ,  0.        ,
          0.        ,  0.00050106,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.

In [135]:
linalg.norm([2,2])

2.8284271247461903